# ATLAS Explorer: Multicore Experiment (Customer Notebook)

Run a multicore experiment with multiple workloads easily—no Python expertise required.

You will:
1. Check project & prior runs
2. Configure / detect credentials
3. Set experiment parameters (ELFs, core type)
4. Run the experiment
5. View Total Cycles and summary table
6. (Optional) Explore derived metrics

Use Shift+Enter to run each cell in order. Modify only the clearly labeled parameter cell.
---

In [1]:
"""Preamble: imports (no edits needed)."""
import os, json, locale, datetime
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
locale.setlocale(locale.LC_ALL, "")
print("Environment initialized.")

Environment initialized.


## 1. Check Project & Existing Experiments
Lists previous runs under `myexperiments/` so you can reuse or compare results later.

In [2]:
# Detect repo root and list existing multicore experiment runs.
from pathlib import Path
import os, datetime

cwd = Path.cwd()
repo_root = None
for p in [cwd, *cwd.parents]:
    if (p / "pyproject.toml").exists() or (p / ".git").exists():
        repo_root = p
        break
repo_root = repo_root or cwd
os.chdir(repo_root)
print(f"Working directory set to repo root: {repo_root}")

exp_root = repo_root / "myexperiments"
rows = []
if exp_root.exists():
    for run_dir in sorted(exp_root.iterdir()):
        if run_dir.is_dir():
            summary = next(run_dir.rglob("reports/summary/summary.json"), None)
            mtime = datetime.datetime.fromtimestamp(run_dir.stat().st_mtime)
            rows.append({
                "name": run_dir.name,
                "summary": str(summary) if summary else "-",
                "modified": mtime.isoformat(timespec="seconds"),
            })
    if not rows:
        print("No experiments found yet under 'myexperiments'.")
else:
    print("No 'myexperiments' directory found yet.")

if rows:
    df = pd.DataFrame(rows).sort_values("modified", ascending=False)
    display(df)

Working directory set to repo root: /Users/jschroeder/Documents/code_repos/mips-ae-pylib


,name,summary,modified
1,I8500_(1_thread)_250811_105319,/Users/jschroeder/Documents/code_repos/mips-ae...,2025-08-11T10:53:28
2,I8500_(2_threads)_250808_103012,/Users/jschroeder/Documents/code_repos/mips-ae...,2025-08-08T10:34:36
0,I8500_(1_thread)_250808_102301,/Users/jschroeder/Documents/code_repos/mips-ae...,2025-08-08T10:27:58


## 2. Configure Credentials
Auto-detect or enter your API key, channel, and region if prompted.
Sources checked: env var `MIPS_ATLAS_CONFIG` and config file in your home directory.

(Already covered above – proceed to run the next cell.)

In [3]:
# Credentials detection / entry (run once)
from pathlib import Path
import os, json
from dotenv import load_dotenv

load_dotenv()
CONFIG_ENV = "MIPS_ATLAS_CONFIG"
cfg_file = Path.home() / ".config/mips/atlaspy/config.json"

if os.environ.get(CONFIG_ENV):
    print("Credentials detected from environment variable.")
elif cfg_file.exists():
    try:
        data = json.loads(cfg_file.read_text())
        os.environ[CONFIG_ENV] = f"{data['apikey']}:{data['channel']}:{data['region']}"
        print(f"Credentials loaded from {cfg_file}")
    except Exception as e:
        print(f"Could not parse {cfg_file}: {e}")

if not os.environ.get(CONFIG_ENV):
    print("No saved credentials found. Enter details below then re-run this cell.")
    ae_apikey  = ""  # REQUIRED
    ae_channel = "development"
    ae_region  = ""  # REQUIRED
    persist_to_file = True

    if not ae_apikey or not ae_channel or not ae_region:
        print("Waiting for input... (fill ae_apikey, ae_channel, ae_region above)")
    else:
        os.environ[CONFIG_ENV] = f"{ae_apikey}:{ae_channel}:{ae_region}"
        print("Session credentials set.")
        if persist_to_file:
            cfg_file.parent.mkdir(parents=True, exist_ok=True)
            cfg_file.write_text(json.dumps({"apikey": ae_apikey, "channel": ae_channel, "region": ae_region}, indent=2))
            print(f"Saved to {cfg_file}")
else:
    print("Ready.")

Credentials detected from environment variable.
Ready.


## 3. Multicore Experiment Overview
We'll run multiple workloads on a multicore configuration. Adjust parameters next, then run.

### Prerequisites
- Credentials configured (step 2)
- ELFs present under `resources/`
- No edits needed unless changing workloads or core

In [4]:
# 3.a EDITABLE PARAMETERS (multiple workloads)
elfs = [
    "resources/mandelbrot_rv64_O0.elf",
    "resources/memcpy_rv64.elf",
]
expdir = "myexperiments"
core = "I8500_(2_threads)"
channel = "development"
apikey = None
region = None
verbose = False

# Optional export format: None | 'json' | 'markdown' | 'html' | 'rich-html' | 'zip'
export = None
out = None
print("Parameters set. Modify above as needed.")

Parameters set. Modify above as needed.


In [5]:
# 3.b Environment validation (no edits needed)
import os, locale
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
locale.setlocale(locale.LC_ALL, "")

config_env = os.environ.get("MIPS_ATLAS_CONFIG")
config_file = Path.home() / ".config/mips/atlaspy/config.json"
if not apikey and not config_env and not config_file.exists():
    print("NOTE: Configure credentials before running (see step 2).")
else:
    print("Credentials appear available. Proceed.")

Credentials appear available. Proceed.


In [6]:
# 4. Run the multicore experiment
from atlasexplorer.atlasexplorer import AtlasExplorer, Experiment

print("Starting multicore experiment run...")
aeinst = AtlasExplorer(apikey, channel, region, verbose=verbose)
experiment = Experiment(expdir, aeinst, verbose=verbose)
for e in elfs:
    experiment.addWorkload(os.path.abspath(e))
experiment.setCore(core)
experiment.run()

try:
    total_cycles = experiment.getSummary().getTotalCycles()
    print(f"Total Cycles: {total_cycles}")
except Exception:
    print("Could not retrieve total cycles from summary object.")

from pathlib import Path
exp_path = Path(expdir)
summary_candidates = list(exp_path.rglob("summary/summary.json"))
if not summary_candidates:
    raise FileNotFoundError("No summary.json found under experiment directory")
summary_candidates.sort(key=lambda p: p.stat().st_mtime, reverse=True)
summary_path = summary_candidates[0]
print(f"Latest summary: {summary_path}")

Starting multicore experiment run...
Available versions: latest, ST-2025-07-16-171806
No report directory found:summary
Total Cycles: 257577
Latest summary: myexperiments/I8500_(2_threads)_250811_105432/I8500_(2_threads)_250811_105432/reports/summary/summary.json


In [ ]:
# 5. View summary (tabular)
import json, pandas as pd
with open(summary_path) as f:
    summary_data = json.load(f)

if isinstance(summary_data, dict):
    df = pd.DataFrame([summary_data])
elif isinstance(summary_data, list):
    df = pd.DataFrame(summary_data)
else:
    df = None

if df is not None:
    display(df.head())
else:
    print(summary_data)

print("Displayed summary (first rows if large).")

In [ ]:
# 6.a Load metrics from summary.json
import json, locale, math
from pathlib import Path
import pandas as pd

if not 'summary_path' in globals():
    raise RuntimeError("summary_path not found (run experiment first or set manually)")

with open(summary_path) as f:
    raw = json.load(f)
metrics_root = raw['Statistics']['Summary Performance Report']

rows = []
for key, entry in metrics_root.items():
    if key == 'ordered_keys':
        continue
    rows.append({
        'Metric': key,
        'Value': entry.get('val'),
        'Unit': entry.get('unit') or entry.get('units') or ''
    })
metrics_df = pd.DataFrame(rows)

locale.setlocale(locale.LC_ALL, "")
import math as _math

def fmt(v):
    if isinstance(v, int):
        try:
            return locale.format_string('%d', v, grouping=True)
        except Exception:
            return v
    if isinstance(v, float) and _math.isfinite(v):
        return f"{v:,.4g}"
    return v

metrics_df['Formatted'] = metrics_df['Value'].map(fmt)
metrics_df.head(len(metrics_df))

In [ ]:
# 6.b OPTIONAL: Text filter
filter_text = ''
filtered = metrics_df
if filter_text:
    ft = filter_text.lower()
    filtered = metrics_df[metrics_df['Metric'].str.lower().str.contains(ft)]
filtered.head(len(filtered))

### 6. Explore Metrics (Interactive)
Load and inspect all metrics from the `summary.json`. You can optionally filter by text.